# Analyzing CIA Factbook Data Using SQL

This notebook examines data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth.

In [19]:
!pip install --upgrade pip

Requirement already up-to-date: pip in /home/nbuser/anaconda3_420/lib/python3.5/site-packages (20.1.1)


In [20]:
!pip install ipython-sql

In [21]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Overview of the Data

In [22]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [23]:
%%sql
SELECT * FROM sqlite_sequence;

 * sqlite:///factbook.db
Done.


name,seq
facts,261


In [24]:
%%sql
SELECT * FROM facts LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Summary Statistics

In [25]:
%%sql
SELECT name, MIN(population) AS population 

FROM facts;

 * sqlite:///factbook.db
Done.


name,population
Antarctica,0


In [26]:
%%sql
SELECT name, MAX(population) AS population FROM facts;

 * sqlite:///factbook.db
Done.


name,population
World,7256490011


In [27]:
%%sql
SELECT name, MIN(population_growth) AS population_growth FROM facts;

 * sqlite:///factbook.db
Done.


name,population_growth
Holy See (Vatican City),0.0


In [28]:
%%sql
SELECT name, MAX(population_growth) AS population_growth FROM facts;

 * sqlite:///factbook.db
Done.


name,population_growth
South Sudan,4.02


## Exploring Average Population and Area

In [29]:
%%sql
SELECT 
    AVG(population) AS avg_population,
    AVG(area) AS avg_area
FROM facts;

 * sqlite:///factbook.db
Done.


avg_population,avg_area
62094928.32231405,555093.546184739


## Finding Densely Populated Countries

In [30]:
%%sql
SELECT 
    name 
FROM facts 
WHERE population > (SELECT AVG(population) from facts)
AND area < (SELECT AVG(area) from facts)

 * sqlite:///factbook.db
Done.


name
Bangladesh
Germany
Japan
Philippines
Thailand
United Kingdom
Vietnam


## Countries With the Highest Ratio of Water to Land

In [31]:
%%sql
SELECT name, CAST(area_water AS FLOAT)/CAST(area_land AS FLOAT) as ratio
FROM facts
ORDER BY ratio DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623


## Countries That Will Add the Most People to Thier Populations Next Year

In [32]:
%%sql
SELECT name, population, population_growth, population * population_growth as new_people
FROM facts
ORDER BY new_people DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population,population_growth,new_people
World,7256490011,1.08,7837009211.88
India,1251695584,1.22,1527068612.48
China,1367485388,0.45,615368424.6
Nigeria,181562056,2.45,444827037.20000005
Pakistan,199085847,1.46,290665336.62


## Countries With Higher Death Rate Than Birth Rate

In [33]:
%%sql
SELECT name, death_rate, birth_rate
FROM facts
WHERE death_rate > birth_rate;

 * sqlite:///factbook.db
Done.


name,death_rate,birth_rate
Austria,9.42,9.41
Belarus,13.36,10.7
Bosnia and Herzegovina,9.75,8.87
Bulgaria,14.44,8.92
Croatia,12.18,9.45
Czech Republic,10.34,9.63
Estonia,12.4,10.51
Germany,11.42,8.47
Greece,11.09,8.66
Hungary,12.73,9.16
